<a href="https://colab.research.google.com/github/Rastaban19/Temporary/blob/main/ML22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer,StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the data
path = "/content/drive/MyDrive/Calculator/csvwmargins.csv"
df = pd.read_csv(path)
df

,QUOTATION,Product,Description,Article size.1,Article size.2,Article size.3,Coverage,Coverage_1,Quantity,Weight kg,...,unit_price_with_margin,Good,Bad,samples,Description.1,certification,material,colour,digital_printing,print_type
0,?Q543738,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown...",22768CCB [3340] Art570432],12.0,3.0,8.0,80%,80%,50,1.266,...,3.771275,Good,Bad,113 Samples,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown...",FSC Mix,NaN,Brown,NaN,InkJet CMYK\n
1,?Q543738,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown...",22768CCB [3340] Art570432],12.0,3.0,8.0,80%,80%,75,1.899,...,2.593427,Good,Bad,113 Samples,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown...",FSC Mix,NaN,Brown,NaN,InkJet CMYK\n
2,?Q543738,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown...",22768CCB [3340] Art570432],12.0,3.0,8.0,80%,80%,100,2.532,...,2.093809,Good,Bad,113 Samples,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown...",FSC Mix,NaN,Brown,NaN,InkJet CMYK\n
3,?Q543738,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown...",22768CCB [3340] Art570432],12.0,3.0,8.0,80%,80%,125,3.165,...,1.724744,Good,Bad,113 Samples,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown...",FSC Mix,NaN,Brown,NaN,InkJet CMYK\n
4,?Q543738,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown...",22768CCB [3340] Art570432],12.0,3.0,8.0,80%,80%,150,3.798,...,1.478576,Good,Bad,113 Samples,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown...",FSC Mix,NaN,Brown,NaN,InkJet CMYK\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,?Q544276,"Pizza box with paper pad 40x40x3.5, Brown [3323]",27768 PBWPP [3323] Art570977],40.0,40.0,35.0,80%,80%,150,28.823,...,2.664839,x,Bad,113 Samples,"Product: Pizza box with paper pad 40x40x3.5, B...",NaN,NaN,Brown,Face : Back,InkJet CMYK\n
970,?Q544276,"Pizza box with paper pad 40x40x3.5, Brown [3323]",27768 PBWPP [3323] Art570977],40.0,40.0,35.0,80%,80%,175,33.626,...,2.488999,x,Bad,113 Samples,"Product: Pizza box with paper pad 40x40x3.5, B...",NaN,NaN,Brown,Face : Back,InkJet CMYK\n
971,?Q544276,"Pizza box with paper pad 40x40x3.5, Brown [3323]",27768 PBWPP [3323] Art570977],40.0,40.0,35.0,80%,80%,200,38.430,...,2.296735,x,Bad,113 Samples,"Product: Pizza box with paper pad 40x40x3.5, B...",NaN,NaN,Brown,Face : Back,InkJet CMYK\n
972,?Q544276,"Pizza box with paper pad 40x40x3.5, Brown [3323]",27768 PBWPP [3323] Art570977],40.0,40.0,35.0,80%,80%,225,43.234,...,2.199185,x,Bad,113 Samples,"Product: Pizza box with paper pad 40x40x3.5, B...",NaN,NaN,Brown,Face : Back,InkJet CMYK\n


In [3]:
# Columns to drop
columns_to_drop = ['QUOTATION','Description','Coverage','Coverage_1','Weight kg','Good','Bad','samples','Description.1',]

# Drop the columns
df = df.drop(columns=columns_to_drop)

In [4]:
 df = df.rename(columns={'Article size.1': 'Article_size_1', "Article size.2": "Article_size_2", "Article size.3": "Article_size_3", "unit_price_for_quant in EUR without VAT_no_margin": "unit_price_for_quant_in_EUR_without_VAT_no_margin"})

In [5]:
df['margin_quant_based'] = df['margin_quant_based'].str.rstrip('%').astype('float') / 100

In [6]:
# Check for missing values
print("Missing values in dataset:\n", df.isnull().sum())

Missing values in dataset:
 Product                                                0
Article_size_1                                         0
Article_size_2                                         0
Article_size_3                                         0
Quantity                                               0
unit_weight                                            0
unit_price_for_quant_in_EUR_without_VAT_no_margin      0
total_price_per_quant                                  0
margin_quant_based                                     0
unit_price_with_margin                                 0
certification                                        640
material                                             658
colour                                                 0
digital_printing                                      54
print_type                                             0
dtype: int64


In [7]:
# Separate features and target
X = df.drop(['unit_price_with_margin'], axis=1)
y = df['unit_price_with_margin']

In [8]:
numerical_features = [x for x, dtype in zip(X.columns, X.dtypes) if dtype.kind in ["i","f"]]
categorical_features = [x for x, dtype in zip(X.columns, X.dtypes) if dtype.kind not in ["i","f"]]

print("Numerical : " + str(numerical_features))
print("Categorical : " + str(categorical_features))

Numerical : ['Article_size_1', 'Article_size_2', 'Article_size_3', 'Quantity', 'unit_weight', 'unit_price_for_quant_in_EUR_without_VAT_no_margin', 'total_price_per_quant', 'margin_quant_based']
Categorical : ['Product', 'certification', 'material', 'colour', 'digital_printing', 'print_type']


In [9]:
# Processing data: cleaning, fill NaN, and OHE
Processor = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        StandardScaler(with_mean=False),
        KBinsDiscretizer(n_bins=3)), numerical_features),

    (make_pipeline(
        SimpleImputer(strategy="constant", fill_value="Not available"),  # Updated fill_value
        OneHotEncoder(categories="auto", handle_unknown="ignore", sparse_output=False)), categorical_features)
)

In [10]:
scaler = StandardScaler()

regressor = GradientBoostingRegressor(
    learning_rate= 0.1,
    max_depth= 7,
    min_samples_split= 10,
    n_estimators= 150
    )

#{'learning_rate': 0.1,
# 'max_depth': 7,
# 'min_samples_split': 10,
# 'n_estimators': 150}

In [11]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

model_pipe = Pipeline([
    ("Processing features", Processor),
    ("Scaling and standardiye data", scaler),
    ("Regressor", regressor)
])
model_pipe

Pipeline(steps=[('Processing features',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('standardscaler',
                                                                   StandardScaler(with_mean=False)),
                                                                  ('kbinsdiscretizer',
                                                                   KBinsDiscretizer(n_bins=3))]),
                                                  ['Article_size_1',
                                                   'Article_size_2',
                                                   'Article_size_3', 'Quantity',
                                                   'unit_weight',
                                                   'unit_price_fo...
                                                                   SimpleImputer(fill_value='Not '
                                                                                            'available',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Product', 'certification',
                                                   'material', 'colour',
                                                   'digital_printing',
                                                   'print_type'])])),
                ('Scaling and standardiye data', StandardScaler()),
                ('Regressor',
                 GradientBoostingRegressor(max_depth=7, min_samples_split=10,
                                           n_estimators=150))])

In [12]:
model_pipe.fit(X, y)

Pipeline(steps=[('Processing features',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('standardscaler',
                                                                   StandardScaler(with_mean=False)),
                                                                  ('kbinsdiscretizer',
                                                                   KBinsDiscretizer(n_bins=3))]),
                                                  ['Article_size_1',
                                                   'Article_size_2',
                                                   'Article_size_3', 'Quantity',
                                                   'unit_weight',
                                                   'unit_price_fo...
                                                                   SimpleImputer(fill_value='Not '
                                                                                            'available',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Product', 'certification',
                                                   'material', 'colour',
                                                   'digital_printing',
                                                   'print_type'])])),
                ('Scaling and standardiye data', StandardScaler()),
                ('Regressor',
                 GradientBoostingRegressor(max_depth=7, min_samples_split=10,
                                           n_estimators=150))])

In [13]:
y_pred_tot = model_pipe.predict(X)

In [14]:
cross_val_scores = cross_val_score(model_pipe, X, y, cv=5)
print(cross_val_scores)

mean_score = np.mean(cross_val_scores)
std_score = np.std(cross_val_scores)

print(f"Mean R2score cross-validation score:{mean_score:.2f}")
print(f"Standard deviation of cross-validation scores:{std_score:.2f}")

[ 0.81490162 -0.00515417  0.89820763  0.71351963  0.87096193]
Mean R2score cross-validation score:0.66
Standard deviation of cross-validation scores:0.34


NOT RELEVANT ANYMORE AFTER CODE UPDATE

The R² score (coefficient of determination) measures how well the model's predictions match the actual values. An R² score of 1.0 means perfect predictions, while a score of 0.0 means the model does no better than predicting the mean of the target variable.
Mean R² Score: 0.55

The average R² score across the 5 folds is 0.55. This means, on average, your model explains about 55% of the variance in the target variable.
This is a moderate score, suggesting the model has some predictive capability but also leaves a fair amount of variance unexplained. It indicates that there may be room for improvement in the model's performance.
Standard Deviation of R² Scores: 0.10

The standard deviation of 0.10 indicates the spread or variability of the R² scores across the different folds.
A lower standard deviation (close to 0) would mean that the model's performance is consistent across all folds. Here, a standard deviation of 0.10 is relatively low, suggesting that while there is some variation in the model's performance, it is not not extreme. The model's performance is reasonably stable across different subsets of the data.

Moderate Model Performance:

The mean R² score of 0.55 indicates that the model is moderately effective. It can explain 55% of the variance in the target variable, but there's still a significant portion (45%) of variance that it doesn't capture. This could mean that:
The model might benefit from further feature engineering, hyperparameter tuning, or trying different algorithms.
There may be important features or interactions not captured by the current model.

Reasonable Consistency Across Folds:

The standard deviation of 0.10 suggests that the model's performance is reasonably consistent across different data splits. While some variation exists, the model's predictions are generally stable, which is a positive sign.
Range of R² Scores:

The individual fold scores vary between 0.36 and 0.63, indicating some variability in model performance depending on the specific subset of data used. This variability might suggest that:
The dataset could have some noise, outliers, or heterogeneity.
Some data folds might be harder to predict accurately due to differences in the underlying data distribution.

Improve the Model:

Consider experimenting with different algorithms, feature selection, or adding new features that might capture more variance in the target variable.
Hyperparameter tuning using techniques like Grid Search or Random Search can help optimize the model’s performance.
Analyze Data Further:

Check for potential outliers or patterns in the data that could be affecting model performance.
Investigate the features to ensure they are appropriately preprocessed and transformed.
Overall, while the model shows some ability to predict the target variable, there's still room for improvement to achieve a better and more robust performance.

In [15]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2_score(y, y_pred_tot)
mean_squared_error(y,y_pred_tot, squared=False)
mean_squared_error(y,y_pred_tot)
mean_absolute_error(y,y_pred_tot)

628.1180295232007

THIS RESULTS MEANS SOMETHING IS WRONG
The model is explaining about 29.5% of the variance in the target variable. This is a relatively low score, indicating that the model has moderate predictive power but still has a lot of room for improvement.

In [16]:
X.columns

Index(['Product', 'Article_size_1', 'Article_size_2', 'Article_size_3',
       'Quantity', 'unit_weight',
       'unit_price_for_quant_in_EUR_without_VAT_no_margin',
       'total_price_per_quant', 'margin_quant_based', 'certification',
       'material', 'colour', 'digital_printing', 'print_type'],
      dtype='object')

In [17]:
X.iloc[0]

,0
Product,"Cardboard Courier boxes 12?8?3, FSC Mix, Brown..."
Article_size_1,12.0
Article_size_2,3.0
Article_size_3,8.0
Quantity,50
unit_weight,0.02532
unit_price_for_quant_in_EUR_without_VAT_no_margin,3.36721
total_price_per_quant,168.3605
margin_quant_based,0.12
certification,FSC Mix


In [18]:
# # Define the data
# data = {
#     "project_title": ["Cardboard Courier boxes 12?8?3, FSC Mix, Brown"],
#     "quantity": [1000000],
#     "description": ["Cardboard Courier boxes"],
#     "measure1": [12.0],
#     "measure2": [8.0],
#     "measure3": [3.0],
#     "certification": ["FSC Mix"],
#     "colour": ["Brown"],
#     "inventory": [334],
#     "type": ["InkJet CMYK\n"],
# }

# # Create a DataFrame
# input_df = pd.DataFrame(data)

# # Define preprocessing steps
# Processor = make_column_transformer(
#     (make_pipeline(
#         SimpleImputer(strategy="mean"),  # Impute missing values for numerical features
#         KBinsDiscretizer(n_bins=3, encode='onehot-dense')),  # Encode numerical features
#         ['measure1', 'measure2', 'measure3', 'quantity', 'inventory']),  # Numerical features

#     (make_pipeline(
#         SimpleImputer(strategy="constant", fill_value="missing"),  # Impute missing values for categorical features
#         OneHotEncoder(categories="auto", handle_unknown="ignore")),  # Encode categorical features
#         ['project_title', 'description', 'certification', 'colour', 'type'])  # Categorical features
# )

# # Create the model pipeline with a regressor (GradientBoostingRegressor in this example)
# model_pipe = make_pipeline(Processor, GradientBoostingRegressor())

# # Example training data (replace X_train and y_train with your actual data)
# X_train = pd.DataFrame({
#     "project_title": ["Cardboard Courier boxes 12?8?3, FSC Mix, Brown"],
#     "quantity": [500000],
#     "description": ["Cardboard Courier boxes"],
#     "measure1": [10.0],
#     "measure2": [7.0],
#     "measure3": [3.0],
#     "certification": ["FSC Mix"],
#     "colour": ["Brown"],
#     "inventory": [200],
#     "type": ["Offset CMYK"],
# })

# y_train = [300]  # Example target value

# # Fit the pipeline with the training data
# model_pipe.fit(X_train, y_train)

# # Make predictions
# try:
#     res = model_pipe.predict(input_df).astype(int)[0]
#     print(res)
# except ValueError as e:
#     print("Error during prediction:", e)

In [19]:
# Defining the variables

Product =	"Cardboard Courier boxes 12?8?3, FSC Mix, Brown"
Article_size_1	= 12.0
Article_size_2	= 3.0
Article_size_3	= 8.0
Quantity	= 50
unit_weight	= 0.02532
unit_price_for_quant_in_EUR_without_VAT_no_margin	= 3.36721
total_price_per_quant	= 168.3605
margin_quant_based	= 0.12
certification	= "FSC Mix"
material	= "NaN"
colour	= "Brown"
digital_printing	= "NaN"
print_type	= "InkJet CMYK\n"

# Defining the data dictionary
data = {
    "Product": Product,
    "Article_size_1": Article_size_1,
    "Article_size_2": Article_size_2,
    "Article_size_3": Article_size_3,
    "Quantity": Quantity,
    "unit_weight": unit_weight,
    "unit_price_for_quant_in_EUR_without_VAT_no_margin": unit_price_for_quant_in_EUR_without_VAT_no_margin,
    "total_price_per_quant": total_price_per_quant,
    "margin_quant_based": margin_quant_based,
    "certification": certification,
    "material": material,
    "colour": colour,
    "digital_printing": digital_printing,
    "print_type": print_type,
}

# Creating DataFrame
input_df = pd.DataFrame([data])  # Wrapping data in a list to create a single-row DataFrame

# Making predictions
res = model_pipe.predict(input_df).astype(float)[0]

# Outputting result
print(res)


3.6531818450606965


The price should be 3.36721 so the code is not good at predicting the right price.

In [20]:
import joblib

joblib.dump(model_pipe,"bagobag2.pkl")

['bagobag2.pkl']

In [21]:
import joblib

model_pipe = joblib.load("bagobag2.pkl")
print("Model loaded successfully")

Model loaded successfully


In [22]:
model_pipe

Pipeline(steps=[('Processing features',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('standardscaler',
                                                                   StandardScaler(with_mean=False)),
                                                                  ('kbinsdiscretizer',
                                                                   KBinsDiscretizer(n_bins=3))]),
                                                  ['Article_size_1',
                                                   'Article_size_2',
                                                   'Article_size_3', 'Quantity',
                                                   'unit_weight',
                                                   'unit_price_fo...
                                                                   SimpleImputer(fill_value='Not '
                                                                                            'available',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Product', 'certification',
                                                   'material', 'colour',
                                                   'digital_printing',
                                                   'print_type'])])),
                ('Scaling and standardiye data', StandardScaler()),
                ('Regressor',
                 GradientBoostingRegressor(max_depth=7, min_samples_split=10,
                                           n_estimators=150))])

In [23]:
# import streamlit as st
# import joblib
# import pandas as pd

# # Define the maximum allowed values
# MAX_QUANTITY = 1000000
# MAX_MEASURE = 150
# DEFAULT_INVENTORY = 10  # Default value for inventory
# DEFAULT_CATEGORICAL = ''  # Default value for categorical inputs

# # Load your model pipeline
# model_pipe = joblib.load("bagobag.pkl")

# # Function to prepare input data
# def prepare_input_data(quantity, measure1, measure2, measure3, inventory, project_title, description, colour, type_):
#     # Create a DataFrame with all features
#     input_data = pd.DataFrame({
#         'quantity': [quantity],
#         'measure1': [measure1],
#         'measure2': [measure2],
#         'measure3': [measure3],
#         'inventory': [inventory],
#         'project_title': [project_title],
#         'description': [description],
#         'colour': [colour],
#         'type': [type_]
#     })
#     return input_data

# # Function to calculate the price using your pipeline
# def predict_price(quantity, measure1, measure2, measure3, inventory, project_title, description, colour, type_):
#     try:
#         # Prepare the input data in the format expected by the model
#         input_data = prepare_input_data(quantity, measure1, measure2, measure3, inventory, project_title, description, colour, type_)

#         # Use the model to make predictions
#         price = model_pipe.predict(input_data)[0]

#         return price
#     except Exception as e:
#         st.error(f"An error occurred while predicting the price: {e}")
#         return None

# # Streamlit app
# def main():
#     st.title("Box Price Estimator")

#     # Input for quantity
#     quantity = st.number_input("Enter quantity:", min_value=1, max_value=MAX_QUANTITY, value=1)

#     # Inputs for measurements
#     measure1 = st.number_input("Enter measure 1 (in cm):", min_value=1, max_value=MAX_MEASURE, value=1)
#     measure2 = st.number_input("Enter measure 2 (in cm):", min_value=1, max_value=MAX_MEASURE, value=1)
#     measure3 = st.number_input("Enter measure 3 (in cm):", min_value=1, max_value=MAX_MEASURE, value=1)

#     # Input for inventory
#     inventory = st.number_input("Enter inventory:", min_value=0, value=DEFAULT_INVENTORY)

#     # Inputs for categorical features
#     project_title = st.text_input("Enter project title:", value=DEFAULT_CATEGORICAL)
#     description = st.text_input("Enter description:", value=DEFAULT_CATEGORICAL)
#     colour = st.text_input("Enter colour:", value=DEFAULT_CATEGORICAL)
#     type_ = st.text_input("Enter type:", value=DEFAULT_CATEGORICAL)

#     # Calculate and display price
#     if st.button("Estimate Price"):
#         price = predict_price(quantity, measure1, measure2, measure3, inventory, project_title, description, colour, type_)
#         if price is not None:
#             st.write(f"Estimated Price: ${price:.2f}")

# if __name__ == "__main__":
#     main()

In [24]:
# import streamlit as st
# import joblib
# import pandas as pd

# # Define the maximum allowed values
# MAX_QUANTITY = 1000000
# MAX_MEASURE = 150
# DEFAULT_INVENTORY = 10  # Default value for inventory
# DEFAULT_CATEGORICAL = ''  # Default value for categorical inputs

# # Load your model pipeline
# model_pipe = joblib.load("bagobag2.pkl")

# # Function to prepare input data
# def prepare_input_data(quantity, measure1, measure2, measure3, inventory, project_title, description, colour, type_):
#     # Create a DataFrame with all features
#     input_data = pd.DataFrame({
#         'quantity': [quantity],
#         'measure1': [measure1],
#         'measure2': [measure2],
#         'measure3': [measure3],
#         'inventory': [inventory],
#         'project_title': [project_title],
#         'description': [description],
#         'colour': [colour],
#         'type': [type_]
#     })
#     return input_data

# # Function to calculate the price using your pipeline
# def predict_price(quantity, measure1, measure2, measure3, inventory, project_title, description, colour, type_):
#     try:
#         # Prepare the input data in the format expected by the model
#         input_data = prepare_input_data(quantity, measure1, measure2, measure3, inventory, project_title, description, colour, type_)

#         # Use the model to make predictions
#         price = model_pipe.predict(input_data)[0]

#         return price
#     except Exception as e:
#         st.error(f"An error occurred while predicting the price: {e}")
#         return None

# # Streamlit app
# def main():
#     st.title("Box Price Estimator")

#     # Input for quantity
#     quantity = st.number_input("Enter quantity:", min_value=1, max_value=MAX_QUANTITY, value=1)

#     # Inputs for measurements
#     measure1 = st.number_input("Enter measure 1 (in cm):", min_value=1, max_value=MAX_MEASURE, value=1)
#     measure2 = st.number_input("Enter measure 2 (in cm):", min_value=1, max_value=MAX_MEASURE, value=1)
#     measure3 = st.number_input("Enter measure 3 (in cm):", min_value=1, max_value=MAX_MEASURE, value=1)

#     # Input for inventory
#     inventory = st.number_input("Enter inventory:", min_value=0, value=DEFAULT_INVENTORY)

#     # Inputs for categorical features
#     project_title = st.text_input("Enter project title:", value=DEFAULT_CATEGORICAL)
#     description = st.text_input("Enter description:", value=DEFAULT_CATEGORICAL)
#     colour = st.text_input("Enter colour:", value=DEFAULT_CATEGORICAL)
#     type_ = st.text_input("Enter type:", value=DEFAULT_CATEGORICAL)

#     # Calculate and display price
#     if st.button("Estimate Price"):
#         price = predict_price(quantity, measure1, measure2, measure3, inventory, project_title, description, colour, type_)
#         if price is not None:
#             st.write(f"Estimated Price: ${price:.2f}")

# if __name__ == "__main__":
#     main()

In [25]:
print(sklearn.__version__)


1.3.2


In [27]:
print(pd.__version__)


2.1.4
